# 05 - Cálculo de Scores Overall

Este notebook realiza:
1. Cálculo do score ponderado por posição
2. Cálculo de scores por categoria (CLASSIFICACAO RANKING)
3. Geração de rankings

In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

BASE_DIR = Path("c:/jobs/botafogo/v3")
OUTPUT_DIR = BASE_DIR / "bases" / "outputs"

## 1. Carregar Dados

In [ ]:
# Carregar dados normalizados
df = pd.read_parquet(OUTPUT_DIR / "_temp_scouts_normalized.parquet")
print(f"Jogadores carregados: {len(df)}")

# Carregar mapeamento de pesos
with open(OUTPUT_DIR / "_temp_weights_map.json", "r") as f:
    weights_dict = json.load(f)
print(f"Indicadores com pesos: {len(weights_dict)}")

# Carregar lista de indicadores disponíveis
with open(OUTPUT_DIR / "_temp_indicators_available.json", "r") as f:
    indicadores_disponiveis = json.load(f)
print(f"Indicadores disponíveis: {len(indicadores_disponiveis)}")

# Carregar pesos para categorias
df_weights = pd.read_parquet(OUTPUT_DIR / "_temp_weights_active.parquet")
print(f"Linhas de pesos: {len(df_weights)}")

In [ ]:
# Verificar posições disponíveis
print("\nPosições mapeadas:")
print(df["mapped_position"].value_counts())

## 2. Calcular Score Overall por Posição

In [ ]:
# Posições para cálculo
POSITIONS = ["GK", "RCB", "LCB", "CB", "RB", "LB", "DM", "CM", "AM", "LW", "RW", "CF"]

def calculate_overall_score(row, weights_dict, position):
    """
    Calcula o score overall ponderado para um jogador em uma posição.
    
    Args:
        row: linha do DataFrame com dados do jogador
        weights_dict: dicionário com pesos por indicador e posição
        position: posição para aplicar os pesos
    
    Returns:
        float: score ponderado (0-100)
    """
    if position not in POSITIONS:
        return np.nan
    
    total_weight = 0
    weighted_sum = 0
    
    for indicador, pos_weights in weights_dict.items():
        norm_col = f"{indicador}_norm"
        
        # Verificar se a coluna normalizada existe e tem valor
        if norm_col not in row.index:
            continue
            
        value = row[norm_col]
        if pd.isna(value):
            continue
        
        # Obter peso para a posição
        weight = pos_weights.get(position, 0)
        if weight == 0:
            continue
        
        weighted_sum += value * weight
        total_weight += weight
    
    if total_weight == 0:
        return np.nan
    
    return weighted_sum / total_weight

In [ ]:
# Calcular score para cada jogador usando sua posição mapeada
print("Calculando scores overall...")

scores = []
for idx, row in df.iterrows():
    position = row["mapped_position"]
    score = calculate_overall_score(row, weights_dict, position)
    scores.append(score)
    
    if (idx + 1) % 1000 == 0:
        print(f"  Processados {idx + 1} jogadores...")

df["overall_score"] = scores
print(f"\nConcluído! Scores calculados para {len(df)} jogadores")

In [ ]:
# Verificar resultado
print("Estatísticas do overall_score:")
print(df["overall_score"].describe())

print(f"\nJogadores com score válido: {df['overall_score'].notna().sum()}")
print(f"Jogadores sem score: {df['overall_score'].isna().sum()}")

## 3. Calcular Scores por Categoria

In [ ]:
# Obter categorias disponíveis
categorias = df_weights["CLASSIFICACAO RANKING"].dropna().unique()
print(f"Categorias disponíveis: {categorias.tolist()}")

In [ ]:
# Criar mapeamento indicador -> categoria
indicador_categoria = dict(zip(
    df_weights["INDICADOR"].str.strip(),
    df_weights["CLASSIFICACAO RANKING"]
))

# Agrupar indicadores por categoria
categorias_indicadores = {}
for indicador in indicadores_disponiveis:
    categoria = indicador_categoria.get(indicador)
    if categoria:
        if categoria not in categorias_indicadores:
            categorias_indicadores[categoria] = []
        categorias_indicadores[categoria].append(indicador)

for cat, inds in categorias_indicadores.items():
    print(f"{cat}: {len(inds)} indicadores")

In [ ]:
def calculate_category_score(row, categoria, indicadores, weights_dict, position):
    """
    Calcula o score ponderado para uma categoria específica.
    """
    if position not in POSITIONS:
        return np.nan
    
    total_weight = 0
    weighted_sum = 0
    
    for indicador in indicadores:
        norm_col = f"{indicador}_norm"
        
        if norm_col not in row.index:
            continue
            
        value = row[norm_col]
        if pd.isna(value):
            continue
        
        weight = weights_dict.get(indicador, {}).get(position, 0)
        if weight == 0:
            continue
        
        weighted_sum += value * weight
        total_weight += weight
    
    if total_weight == 0:
        return np.nan
    
    return weighted_sum / total_weight

In [ ]:
# Calcular score por categoria
print("Calculando scores por categoria...")

for categoria, indicadores in categorias_indicadores.items():
    col_name = f"score_{categoria.replace(' ', '_').lower()}"
    print(f"  Calculando {col_name}...")
    
    scores_cat = []
    for idx, row in df.iterrows():
        position = row["mapped_position"]
        score = calculate_category_score(row, categoria, indicadores, weights_dict, position)
        scores_cat.append(score)
    
    df[col_name] = scores_cat

print("\nScores por categoria calculados!")

## 4. Gerar Rankings

In [ ]:
# Ranking geral (todos os jogadores)
df["rank_overall"] = df["overall_score"].rank(ascending=False, method="min")
print(f"Ranking geral calculado")

# Ranking por posição
df["rank_position"] = df.groupby("mapped_position")["overall_score"].rank(ascending=False, method="min")
print(f"Ranking por posição calculado")

In [ ]:
# Top 10 overall
print("\nTop 10 Overall:")
top_cols = ["player_name", "mapped_position", "overall_score", "rank_overall", "rank_position"]
available_cols = [c for c in top_cols if c in df.columns]
df.nsmallest(10, "rank_overall")[available_cols]

In [ ]:
# Top 5 por posição
print("\nTop 5 por Posição:")
for pos in POSITIONS:
    df_pos = df[df["mapped_position"] == pos]
    if len(df_pos) > 0:
        print(f"\n{pos}:")
        top5 = df_pos.nsmallest(5, "rank_position")[["player_name", "overall_score", "rank_position"]]
        for _, row in top5.iterrows():
            print(f"  {int(row['rank_position']):3d}. {row['player_name']}: {row['overall_score']:.2f}")

## 5. Verificar Resultados

In [ ]:
# Estatísticas por posição
print("\nEstatísticas de overall_score por posição:")
stats = df.groupby("mapped_position")["overall_score"].agg(["count", "mean", "std", "min", "max"])
stats

In [ ]:
# Colunas de score criadas
score_cols = [c for c in df.columns if c.startswith("score_") or c == "overall_score"]
print(f"\nColunas de score criadas:")
for col in score_cols:
    valid = df[col].notna().sum()
    print(f"  {col}: {valid} valores válidos")

## 6. Salvar Dados com Scores

In [ ]:
# Salvar DataFrame com scores
df.to_parquet(OUTPUT_DIR / "_temp_scouts_scored.parquet", index=False)
print(f"Dados com scores salvos: {OUTPUT_DIR / '_temp_scouts_scored.parquet'}")

print(f"\nTotal de jogadores: {len(df)}")
print(f"Total de colunas: {len(df.columns)}")